In [9]:
import tkinter as tk
from tkinter import messagebox
import pymongo
from bson import ObjectId
import json

class CustomButton(tk.Button):
    def __init__(self, master=None, **kwargs):
        super().__init__(master, **kwargs)
        self.configure(highlightthickness=0, bd=0, relief=tk.FLAT)
        self.config(fg="#ecf0f1", activeforeground="#ecf0f1")
        self.config(activebackground="#2c3e50", borderwidth=0, relief=tk.FLAT)

def create_gui():
    def create_entry():
        # Get values from entry fields
        General_Health = entries[0].get()
        Checkup = entries[1].get()
        Exercise = entries[2].get()
        Green_Vegetables_Consumption = entries[3].get()
        Fruit_Consumption = entries[4].get()
        Alcohol_Consumption = entries[5].get()
        PatientID = entries[6].get()  # Assuming this is the ID of the doctor
        
        # Check if any entry field is empty
        if not all([General_Health, Checkup, Exercise, Green_Vegetables_Consumption, Fruit_Consumption, Alcohol_Consumption, PatientID]):
            messagebox.showerror("Error", "Please fill in all fields.")
            return

        # Insert patient data with doctor and address references
        health_data = {
            "General_Health": General_Health,
            "Checkup": Checkup,
            "Exercise(kg)": Exercise,
            "Green_Vegetables_Consumption": Green_Vegetables_Consumption,
            "Fruit_Consumption": Fruit_Consumption,
            "Alcohol_Consumption": Alcohol_Consumption,
            "PatientID": ObjectId( PatientID ),  # Convert to ObjectId
            
        }

        result =  health_data_collection.insert_one(health_data)

        messagebox.showinfo("Success", "Entry created successfully.")
    def Aggregation_func():
        query = search_entry[4].get()
        if query:
            try:
                query = json.loads(query)
                result = health_data_collection.aggregate([query])
                messagebox.showinfo("Query Result", f"Query executed: {query}")
                for doc in result:
                    result_text.insert(tk.END, f"{doc}\n\n")
            except Exception as e:
                messagebox.showerror("Error", f"Aggregate Query Error: {str(e)}")
        else:
            messagebox.showerror("Error", "Query not execute")
            
    def create_mapReduce():
        code = search_entry[5].get()
        if code:
            try:
                # Execute the MapReduce operation
                exec(code)
                messagebox.showinfo("Query Result", f"Query executed: {code}")
                # Retrieve and print the results
                result = db.Output_sum_Health_Data.find()
                count = 0
                for doc in result:
                    count += 1
                    result_text.insert(tk.END, f"{doc}\n\n")
                result_label.config(text=f"Search Results / Read Data: ({count})")
            except Exception as e:
                messagebox.showerror("Error", f"MapReduce Query Error: {str(e)}")
        else:
            messagebox.showerror("Error", "not Code to execute")
    
    def execute_query():
        query = search_entry[1].get()  # Get the text from the entry widget
        sort_query = search_entry[2].get()
        if query:
            try:
                query = json.loads(query)
                if sort_query:
                    try:
                        sort_query = eval(sort_query)
                        result =  health_data_collection.find(query).sort(sort_query[0],sort_query[1])
                    except:
                        messagebox.showerror("Error", f"Query Sort Error")
                        return
                else:
                    # Find patients based on the constructed query
                    result =  health_data_collection.find(query)
                
                count =0
                for doc in result:
                    count +=1
                    result_text.insert(tk.END, f"{doc}\n\n")

                result_label.config(text=f"Search Results / Read Data: ({count})")
            except:
                messagebox.showerror("Error", f"Query Error")
                return
            
            # Execute your query and display results (replace this with your actual query logic)
            messagebox.showinfo("Query Result", f"Query executed: {query}")
        elif sort_query:
            try:
                sort_query = eval(sort_query)
                result =  health_data_collection.find().sort(sort_query[0],sort_query[1])
                count =0
                for doc in result:
                    count +=1
                    result_text.insert(tk.END, f"{doc}\n\n")

                result_label.config(text=f"Search Results / Read Data: ({count})")
            except:
                messagebox.showerror("Error", f"Query Sort Error")
                return
        else:
            messagebox.showerror("Error", f"Query not Inserted")
    
    def read_entry():
        # Get values from entry fields
        General_Health = entries[0].get()
        Checkup = entries[1].get()
        Exercise = entries[2].get()
        Green_Vegetables_Consumption = entries[3].get()
        Fruit_Consumption = entries[4].get()
        Alcohol_Consumption = entries[5].get()
        PatientID = entries[6].get()
        # Display read data in the results_frame
        result_text.delete(1.0, tk.END)  # Clear previous content

        # Construct query based on filled entries
        query = {}
        if General_Health:
            query["General_Health"] = General_Health
        if Checkup:
            query["Checkup"] = Checkup
        if Exercise:
            query["Exercise"] = Exercise
        if Green_Vegetables_Consumption:
            query["Green_Vegetables_Consumption)"] = Green_Vegetables_Consumption
        if Fruit_Consumption:
            query["Fruit_Consumption"] = Fruit_Consumption
        if Alcohol_Consumption:
            query["Alcohol_Consumption"] = Alcohol_Consumption
        if PatientID:
            query["PatientID"] = ObjectId(PatientID)

        # Find patients based on the constructed query
        result =  health_data_collection.find(query)
        count =0
        for doc in result:
            count +=1
            result_text.insert(tk.END, f"{doc}\n\n")
            
        result_label.config(text=f"Search Results / Read Data: ({count})")


    def search_entry():
        ID = search_entry[0].get()
        try:
            result =  health_data_collection.find({"_id": ObjectId(ID)})
            # Get the first document (assuming there's only one)
            pat = result[0]

            # Populate the GUI form fields with patient data
            entries[0].insert(0, pat["General_Health"])
            entries[1].insert(0, pat["Checkup"])
            entries[2].insert(0, pat["Exercise"])
            entries[3].insert(0, pat["Green_Vegetables_Consumption"])
            entries[4].insert(0, pat["Fruit_Consumption"])
            entries[5].insert(0, pat["Alcohol_Consumption"])
            entries[6].insert(0,str(pat["PatientID"]))
            
            # Display search results
            result_text.delete(1.0, tk.END)  # Clear previous content
            result_text.insert(tk.END, f"Search Results for ID {ID}:\n{pat}")
        except:
            messagebox.showerror("Error", "Health_Data not found.")

    def delete_entry():
        ID = search_entry[0].get()
        try:
            result = health_data_collection.find({"_id": ObjectId(ID)})
            health_data_collection.delete_one({"_id": ObjectId(ID)})
            messagebox.showinfo("Delete", "Delete operation performed.")
        except:
            messagebox.showerror("Error", "Not Deleted.")
        

    def update_entry():
        ID = search_entry[0].get()
        try:
            result = patient_collection.find({"_id": ObjectId(ID)})
            General_Health = entries[0].get()
            Checkup = entries[1].get()
            Exercise = entries[2].get()
            Green_Vegetables_Consumption = entries[3].get()
            Fruit_Consumption = entries[4].get()
            Alcohol_Consumption = entries[5].get()
            PatientID = entries[6].get()
            patient_collection.update_one({"_id": ObjectId(ID)}, {"$set": {"General_Health": General_Health, "Checkup": Checkup, "Exercise": Exercise, "Green_Vegetables_Consumption": Green_Vegetables_Consumption , "Fruit_Consumption": Fruit_Consumption, "Alcohol_Consumption": Alcohol_Consumption,"PatientID":PatientID}})
            messagebox.showinfo("Update", "Update operation performed.")
        except:
            messagebox.showerror("Error", "Not Updated.")
   
    def create_Indexes():
        query = search_entry[3].get()  # Get the text from the entry widget
        if query:
            try:
                health_data_collection.create_index(eval(query))
                messagebox.showinfo("Create Index", "Create Index operation performed.")
            except:
                messagebox.showerror("Error", "Not Indexed.")
        else:
            messagebox.showerror("Error", "Not Query Index is Inserted.")
    def Drop_Indexes():
        name = search_entry[3].get()  # Get the text from the entry widget
        if name:
            try:
                health_data_collection.drop_index(name)
                messagebox.showinfo("Drop_Indexes", f"Drop_Index {name} operation performed.")
            except:
                messagebox.showerror("Error", "Not Indexed.")
        else:
            messagebox.showerror("Error", "Not Index is Dropped.")
    def Show_Indexes():
        result_text.delete(1.0, tk.END)
        result = health_data_collection.list_indexes()
        count = 0
        for doc in result:
            count += 1
            result_text.insert(tk.END, f"{doc}\n\n")

            result_label.config(text=f"Search Results / Read Data: ({count})")
            
    def clear_textboxes():
        result_text.delete(1.0,tk.END)
        for entry in entries:
            entry.delete(0, tk.END)
        for en in search_entry:
            en.delete(0,tk.END)

    root = tk.Tk()
    root.title("CRUD Health_Data")
    root.geometry("900x700")  # Increase window height
    root.configure(bg="#34495e")

    form_frame = tk.Frame(root, borderwidth=2, relief=tk.GROOVE, bg="#34495e")
    form_frame.pack(pady=20, padx=20)  # Increase padding

    fields = ["General_Health:", "Checkup:", "Exercise:", "Green_Vegetables_Consumption:", "Fruit_Consumption:", "Alcohol_Consumption:","PatientID:"]
    entries = []

    for i, field_name in enumerate(fields):
        label = tk.Label(form_frame, text=field_name, font=("Arial", 12, "bold"), bg="#34495e", fg="#ecf0f1")
        label.grid(row=i, column=0, padx=10, pady=5, sticky=tk.W)
        entry = tk.Entry(form_frame, font=("Arial", 12), bg="#2c3e50", fg="#ecf0f1", width=50)  # Adjust width here
        entry.grid(row=i, column=1, padx=10, pady=5)
        entries.append(entry)


    button_frame = tk.Frame(root, bg="#34495e")
    button_frame.pack(side=tk.RIGHT, padx=20, pady=8)  # Increase padding

    create_btn = CustomButton(button_frame, text="Create", font=("Arial", 12, "bold"), command=create_entry, bg="#DEB887", relief=tk.RAISED)
    create_btn.pack(pady=5, anchor="e", fill=tk.X, ipadx=7, ipady=2)
    read_btn = CustomButton(button_frame, text="Read", font=("Arial", 12, "bold"), command=read_entry, bg="#DEB887", relief=tk.RAISED)
    read_btn.pack(pady=5, anchor="e", fill=tk.X, ipadx=7, ipady=2)
    search_btn = CustomButton(button_frame, text="Search", font=("Arial", 12, "bold"), command=search_entry, bg="#DEB887", relief=tk.RAISED)
    search_btn.pack(pady=5, anchor="e", fill=tk.X, ipadx=7, ipady=2)
    execute_button = CustomButton(button_frame, text="Execute Query", font=("Arial", 12, "bold"), command=execute_query, bg="#DEB887", relief=tk.RAISED)
    execute_button.pack(pady=5, anchor="e", fill=tk.X, ipadx=7, ipady=2)
    update_btn = CustomButton(button_frame, text="Update", font=("Arial", 12, "bold"), command=update_entry, bg="#DEB887", relief=tk.RAISED)
    update_btn.pack(pady=5, anchor="e", fill=tk.X, ipadx=7, ipady=2)
    delete_btn = CustomButton(button_frame, text="Delete", font=("Arial", 12, "bold"), command=delete_entry, bg="#DEB887", relief=tk.RAISED)
    delete_btn.pack(pady=5, anchor="e", fill=tk.X, ipadx=7, ipady=2)
    clear_button = CustomButton(button_frame, text="Clear", font=("Arial", 12, "bold"), command=clear_textboxes, bg="#DEB887", relief=tk.RAISED)
    clear_button.pack(pady=5, anchor="e", fill=tk.X, ipadx=7, ipady=2)
    Indexes_button = CustomButton(button_frame, text="CreateIndexes", font=("Arial", 12, "bold"), command=create_Indexes, bg="#DEB887", relief=tk.RAISED)
    Indexes_button.pack(pady=5, anchor="e", fill=tk.X, ipadx=7, ipady=2)
    Drop_button = CustomButton(button_frame, text="Drop_Indexes", font=("Arial", 12, "bold"), command=Drop_Indexes, bg="#DEB887", relief=tk.RAISED)
    Drop_button.pack(pady=5, anchor="e", fill=tk.X, ipadx=7, ipady=2)
    Show_button = CustomButton(button_frame, text="Show_Indexes", font=("Arial", 12, "bold"), command=Show_Indexes, bg="#DEB887", relief=tk.RAISED)
    Show_button.pack(pady=5, anchor="e", fill=tk.X, ipadx=7, ipady=2)
    Aggregation_func = CustomButton(button_frame, text="Aggregation func", font=("Arial", 12, "bold"), command=Aggregation_func, bg="#DEB887", relief=tk.RAISED)
    Aggregation_func.pack(pady=5, anchor="e", fill=tk.X, ipadx=7, ipady=2)
    create_mapReduce = CustomButton(button_frame, text="create_mapReduce", font=("Arial", 12, "bold"), command=create_mapReduce, bg="#DEB887", relief=tk.RAISED)
    create_mapReduce.pack(pady=5, anchor="e", fill=tk.X, ipadx=7, ipady=2)
    
    
    search_frame = tk.Frame(root, borderwidth=2, relief=tk.GROOVE, bg="#34495e")
    search_frame.pack(pady=20, padx=20, fill=tk.X)  # Increase padding



    fields = ["Search by Id:", "Query: ", "Sort Query by:", "CreateIndexes :", "Aggregation func:", "Map_Reduce:"]
    search_entry = []

    for i, field_name in enumerate(fields):
        label = tk.Label(search_frame, text=field_name, font=("Arial", 12, "bold"), bg="#34495e", fg="#ecf0f1")
        label.grid(row=i, column=0, padx=10, pady=5, sticky=tk.W)
        entry = tk.Entry(search_frame, font=("Arial", 12), bg="#2c3e50", fg="#ecf0f1", width=60)  # Adjust width here
        entry.grid(row=i, column=1, padx=10, pady=5)
        search_entry.append(entry)

    results_frame = tk.Frame(root, borderwidth=2, relief=tk.GROOVE, bg="#34495e")
    results_frame.pack(pady=20, padx=20, fill=tk.BOTH, expand=True)  # Increase padding

    result_label = tk.Label(results_frame, text="Search Results / Read Data:", font=("Arial", 12, "bold"), bg="#34495e", fg="#ecf0f1")
    result_label.pack(pady=5, padx=10, anchor="w")

    result_text = tk.Text(results_frame, font=("Arial", 12), height=10, bg="#2c3e50", fg="#ecf0f1")
    result_text.pack(pady=5, padx=10, fill=tk.BOTH, expand=True)

    root.mainloop()

# To run in a standalone Python script
if __name__ == "__main__":
    # Replace with your MongoDB connection string if different from localhost
    mongo_uri = "mongodb://localhost:27017/"

    # Connect to MongoDB
    client = pymongo.MongoClient(mongo_uri)

    # Access the database (replace "projectBigData" if needed)
    db = client["projectBigData"]
    patient_collection = db["Patient"]
    health_data_collection = db["Health_Data"]
    diseases_collection = db["disease"]
    diabetes_collection = db["diabetes"]
    arthritis_collection = db["arthritis"]
    doctor_collection = db["Doctors"]
    address_collection = db["Address"]
    create_gui()
